In [ ]:
import pandas as pd
import numpy as np
from numpy import log as ln
import math

In [ ]:
nilai_siswa = [[None,1,1,1,1,1,1,1,0,1],
                      [1,1,1,1,1,1,1,0,1,0],
                      [1,1,1,1,1,1,0,1,0,0],
                      [1,1,1,1,1,1,0,1,0,0],
                      [1,1,1,1,1,1,0,1,0,0],
                      [1,1,1,1,1,0,1,0,0,0],
                      [1,1,1,1,0,1,0,0,0,0],
                      [1,0,1,0,1,0,0,0,0,0],
                      [0,1,0,1,0,0,0,0,0,0]]
nilai = pd.DataFrame(nilai_siswa)
nilai

,0,1,2,3,4,5,6,7,8,9
0,NaN,1,1,1,1,1,1,1,0,1
1,1.0,1,1,1,1,1,1,0,1,0
2,1.0,1,1,1,1,1,0,1,0,0
3,1.0,1,1,1,1,1,0,1,0,0
4,1.0,1,1,1,1,1,0,1,0,0
5,1.0,1,1,1,1,0,1,0,0,0
6,1.0,1,1,1,0,1,0,0,0,0
7,1.0,0,1,0,1,0,0,0,0,0
8,0.0,1,0,1,0,0,0,0,0,0


In [ ]:
jumlah_baris, jumlah_kolom = nilai.shape

print(f"Jumlah Baris: {jumlah_baris}")
print(f"Jumlah Kolom: {jumlah_kolom}")

Jumlah Baris: 9
Jumlah Kolom: 10


In [ ]:
#iterasi 0
item_proportion = nilai.mean(axis = 0)
nilai.loc["item proportion"] = item_proportion

difficulty = (1-item_proportion) / item_proportion
dlogit = np.log(difficulty)
nilai.loc['difficulty logit'] = dlogit

person_proportion = nilai.mean(axis = 1)
nilai['person proportion'] = person_proportion

ability = person_proportion/(1-person_proportion)
alogit = np.log(ability)
nilai["ability logit"] = alogit

adjd = nilai[-1:] - nilai[-1:]['person proportion'][0]
adjd.rename(index={'difficulty logit': 'adjust difficulty'}, inplace=True)

nilai = pd.concat([nilai, adjd])

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
display(nilai)

,0,1,2,3,4,5,6,7,8,9,person proportion,ability logit
0,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.888889,2.079442
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.800000,1.386294
2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.700000,0.847298
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.700000,0.847298
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.700000,0.847298
5,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.600000,0.405465
6,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000
7,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.300000,-0.847298
8,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,-1.386294
item proportion,0.875000,0.888889,0.888889,0.888889,0.777778,0.666667,0.333333,0.444444,0.111111,0.111111,0.598611,0.399681


In [ ]:
# Inisialisasi nilai sum_squared dari iterasi sebelumnya
previous_sum_squared = float('inf')  # Nilai awal bisa diatur sesuai kebutuhan

# Set threshold
threshold = 1e-3
iteration = 1
while True:

    # Calculate expected values
    expected_data = pd.DataFrame(
        [[(np.exp(alogit[x] - adjd[y]) / (1 + np.exp(alogit[x] - adjd[y])))[0] for y in range(jumlah_kolom)] for x in range(jumlah_baris)])

    print(f'\nIterasi ke-{iteration}')

    print('\nExpected Values:')
    display(expected_data)
    # Calculate variances
    var = [(expected_data.iloc[x] * (1 - expected_data.iloc[x])) for x in range(jumlah_baris)]
    var_expected_data = pd.DataFrame(var)

    # Calculate residuals
    residual = nilai - expected_data
    residual = residual.drop(['adjust difficulty', 'difficulty logit', 'item proportion'], axis=0)
    residual = residual.drop(['person proportion', 'ability logit'], axis=1)

    sum_var_col = (-1) * var_expected_data.sum(axis=0)
    var_expected_data.loc["Vertical Sum of Variances"] = sum_var_col
    sum_var_row = (-1) * var_expected_data.sum(axis=1)
    var_expected_data['Horizontal Sum of Variances'] = sum_var_row

    sum_of_residual_col = residual.loc['Sum of Residuals Column'] = -1 * (residual.sum(axis=0))
    sum_of_residual_row = residual['Sum of Residuals Row'] = residual.sum(axis=1)

    squared_value = sum_of_residual_row[:-1] ** 2
    sum_squared = squared_value.sum()

    res_sq_val = residual['Sum of Residuals Row'][-1:]
    residual['Sum of Residuals Row'] = residual['Sum of Residuals Row'].replace([res_sq_val], sum_squared)

    alogit = residual['New Ability Logit'] = alogit - (sum_of_residual_row / sum_var_row)

    dlogit = adjd - (sum_of_residual_col / sum_var_col)
    dlogit = dlogit.rename(index={'adjust difficulty': 'New Difficulty Logit'})
    dlogit = dlogit.rename(columns={'ability logit': 'Sum of Residuals Row', 'person proportion': 'New Ability Logit'})

    avg_dlogit = dlogit[-1:].mean(axis=1)
    dlogit['Sum of Residuals Row'] = avg_dlogit

    residual = pd.concat([residual, dlogit])

    adjd = residual[-1:] - residual[-1:]['Sum of Residuals Row'][0]
    adjd = adjd.rename(index={'New Difficulty Logit': 'Adjusted New Difficulty Logit'})

    residual = pd.concat([residual, adjd])

    print('\nVariance of Expected Values:')
    display(var_expected_data)

    print('\nResiduals:')
    display(residual)
    if abs(sum_squared - previous_sum_squared) < threshold:
        print(f"Optimization converged. Stopping iterations at iteration {iteration}.")
        break
    iteration += 1
    # Menyimpan nilai sum_squared untuk iterasi berikutnya
    previous_sum_squared = sum_squared



Iterasi ke-1

Expected Values:


,0,1,2,3,4,5,6,7,8,9
0,0.971247,0.974751,0.974751,0.974751,0.944102,0.906115,0.706987,0.794261,0.376250,0.376250
1,0.944102,0.950745,0.950745,0.950745,0.894123,0.828345,0.546775,0.658733,0.231716,0.231716
2,0.907854,0.918433,0.918433,0.918433,0.831257,0.737874,0.413056,0.529630,0.149613,0.149613
3,0.907854,0.918433,0.918433,0.918433,0.831257,0.737874,0.413056,0.529630,0.149613,0.149613
4,0.907854,0.918433,0.918433,0.918433,0.831257,0.737874,0.413056,0.529630,0.149613,0.149613
5,0.863642,0.878618,0.878618,0.878618,0.760009,0.644080,0.311486,0.419902,0.101609,0.101609
6,0.808519,0.828345,0.828345,0.828345,0.678583,0.546775,0.231716,0.325493,0.070114,0.070114
7,0.644080,0.674069,0.674069,0.674069,0.475013,0.340819,0.114463,0.171371,0.031303,0.031303
8,0.513526,0.546775,0.546775,0.546775,0.345466,0.231716,0.070114,0.107654,0.018501,0.018501



Variance of Expected Values:


,0,1,2,3,4,5,6,7,8,9,Horizontal Sum of Variances
0,0.027926,0.024612,0.024612,0.024612,0.052773,0.085071,0.207156,0.163411,0.234686,0.234686,-1.079544
1,0.052773,0.046829,0.046829,0.046829,0.094667,0.142189,0.247812,0.224804,0.178024,0.178024,-1.258780
2,0.083655,0.074914,0.074914,0.074914,0.140269,0.193416,0.242441,0.249122,0.127229,0.127229,-1.388102
3,0.083655,0.074914,0.074914,0.074914,0.140269,0.193416,0.242441,0.249122,0.127229,0.127229,-1.388102
4,0.083655,0.074914,0.074914,0.074914,0.140269,0.193416,0.242441,0.249122,0.127229,0.127229,-1.388102
5,0.117764,0.106648,0.106648,0.106648,0.182395,0.229241,0.214463,0.243584,0.091285,0.091285,-1.489961
6,0.154816,0.142189,0.142189,0.142189,0.218108,0.247812,0.178024,0.219547,0.065198,0.065198,-1.575272
7,0.229241,0.219700,0.219700,0.219700,0.249376,0.224661,0.101361,0.142003,0.030323,0.030323,-1.666389
8,0.249817,0.247812,0.247812,0.247812,0.226119,0.178024,0.065198,0.096064,0.018159,0.018159,-1.594977
Vertical Sum of Variances,-1.083303,-1.012532,-1.012532,-1.012532,-1.444245,-1.687247,-1.741336,-1.836780,-0.999361,-0.999361,12.829230



Residuals:


...


Variance of Expected Values:


,0,1,2,3,4,5,6,7,8,9,Horizontal Sum of Variances
0,0.000595,0.000594,0.000594,0.000594,0.002448,0.007750,0.084185,0.042257,0.247745,0.247745,-0.634506
1,0.002437,0.002435,0.002435,0.002435,0.009917,0.030384,0.208450,0.134310,0.175617,0.175617,-0.744036
2,0.008722,0.008713,0.008713,0.008713,0.034125,0.094010,0.239189,0.241568,0.069422,0.069422,-0.782596
3,0.008722,0.008713,0.008713,0.008713,0.034125,0.094010,0.239189,0.241568,0.069422,0.069422,-0.782596
4,0.008722,0.008713,0.008713,0.008713,0.034125,0.094010,0.239189,0.241568,0.069422,0.069422,-0.782596
5,0.028677,0.028646,0.028646,0.028646,0.099246,0.204641,0.134735,0.208844,0.022578,0.022578,-0.807236
6,0.078490,0.078415,0.078415,0.078415,0.201397,0.247092,0.054953,0.106002,0.007554,0.007554,-0.938284
7,0.227711,0.227633,0.227633,0.227633,0.213703,0.107746,0.010549,0.022735,0.001330,0.001330,-1.268002
8,0.248081,0.248106,0.248106,0.248106,0.140307,0.056106,0.004836,0.010569,0.000604,0.000604,-1.205425
Vertical Sum of Variances,-0.612158,-0.611965,-0.611965,-0.611965,-0.769390,-0.935750,-1.215274,-1.249422,-0.663693,-0.663693,7.945277



Residuals:


,0,1,2,3,4,5,6,7,8,9,Sum of Residuals Row,New Ability Logit
0,NaN,0.000594,0.000594,0.000594,0.002454,0.007811,0.092796,0.044212,-0.547487,0.452513,0.054080,4.714737
1,0.002443,0.002441,0.002441,0.002441,0.010017,0.031368,0.296162,-0.840133,0.772732,-0.227268,0.052644,3.285932
2,0.008800,0.008790,0.008790,0.008790,0.035376,0.105045,-0.396023,0.408176,-0.075055,-0.075055,0.037632,1.975558
3,0.008800,0.008790,0.008790,0.008790,0.035376,0.105045,-0.396023,0.408176,-0.075055,-0.075055,0.037632,1.975558
4,0.008800,0.008790,0.008790,0.008790,0.035376,0.105045,-0.396023,0.408176,-0.075055,-0.075055,0.037632,1.975558
5,0.029550,0.029517,0.029517,0.029517,0.111730,-0.712976,0.839508,-0.297132,-0.023112,-0.023112,0.013006,0.711078
6,0.085862,0.085772,0.085772,0.085772,-0.720462,0.553929,-0.058359,-0.120529,-0.007611,-0.007611,-0.017467,-0.450090
7,0.350705,-0.649557,0.350443,-0.649557,0.690519,-0.122834,-0.010662,-0.023277,-0.001332,-0.001332,-0.066886,-2.233522
8,-0.456196,0.543518,-0.456482,0.543518,-0.168800,-0.059666,-0.004860,-0.010683,-0.000604,-0.000604,-0.070859,-3.031165
Sum of Residuals Column,-0.038764,-0.038653,-0.038653,-0.038653,-0.031585,-0.012765,0.033485,0.023015,0.032580,0.032580,0.019914,NaN


Optimization converged. Stopping iterations at iteration 9.


In [ ]:
print("Final Result")
print("CELL FIT")
residual_final = residual.drop(['Sum of Residuals Column', 'Adjusted New Difficulty Logit', 'Adjusted New Difficulty Logit'], axis=0)
residual_final = residual_final.drop(['Sum of Residuals Row',	'New Ability Logit'], axis=1)
residual_final

var_expected_data_final=var_expected_data.drop(['Horizontal Sum of Variances'], axis=1)
var_expected_data_final=var_expected_data_final.drop(['Vertical Sum of Variances'], axis=0)


cell_fit = pd.DataFrame((nilai - expected_data)**2/var_expected_data_final)
cell_fit = cell_fit.drop([ 'ability logit', 'person proportion'], axis=1)
cell_fit = cell_fit.drop(['adjust difficulty', 'difficulty logit', 'item proportion'], axis=0)


#print OutFit Item
OutFit_item = cell_fit.mean(axis = 0)
cell_fit.loc["OutFit Item"] = OutFit_item

#print inFit Item
# Menghitung SUMSQ
result_sumsq_item = np.sum(np.square(residual_final), axis=0)
# Menghitung SUM
result_sum_item = (-1)*sum_var_col
# Menghitung ekspresi yang diberikan
InFit_item = result_sumsq_item / result_sum_item
# Menambahkan nilai InFit ke dalam DataFrame cell_fit
cell_fit.loc["InFit Item"] = InFit_item

#print OutFit Person
OutFit_person = cell_fit.iloc[:-2].mean(axis = 1)
cell_fit["OutFit Person"] = OutFit_person


#print inFit Person
# Menghitung SUMSQ
result_sumsq_person = np.sum(np.square(residual_final), axis=1)
# Menghitung SUM
result_sum_person = (-1)*sum_var_row
# Menghitung ekspresi yang diberikan
InFit_person = result_sumsq_person / result_sum_person
# Menambahkan nilai InFit ke dalam DataFrame cell_fit
cell_fit["InFit Person"] = InFit_person

#print
display(cell_fit)


print("EXPECTED VALUES FROM LAST ITERATION")
combined_data=pd.concat([expected_data, pd.DataFrame({'Final Person Ability Logit': alogit})], axis=1)
columns_to_drop = ['Sum of Residuals Column', 'Vertical Sum of Variances', 'difficulty logit', 'item proportion']
expected_data_final = combined_data.drop(index=columns_to_drop, errors='ignore')

adjd = adjd.rename(index={'Adjusted New Difficulty Logit': 'Final Item Difficulty Logit'})
combined_data=pd.concat([expected_data_final, pd.DataFrame(adjd)], axis=0)
columns_to_drop = ['Sum of Residuals Row', 'New Ability Logit']
expected_data_final = combined_data.drop(columns=columns_to_drop, errors='ignore')

#print Item Standard Error
Item_Standard_Error = np.sqrt(-1/sum_var_col)
expected_data_final.loc["Item Standard Error"] = Item_Standard_Error

#print Person Standard Error
Person_Standard_Error = np.sqrt(-1/sum_var_row)
expected_data_final["Person Standard Error"] = Person_Standard_Error

display(expected_data_final)

Final Result
CELL FIT


,0,1,2,3,4,5,6,7,8,9,OutFit Person,InFit Person
0,NaN,0.000595,0.000595,0.000595,0.002460,0.007872,0.102287,0.046257,1.209882,0.826527,0.244119,0.811882
1,0.002449,0.002447,0.002447,0.002447,0.010118,0.032384,0.420781,5.255202,3.400095,0.294109,0.942248,1.939972
2,0.008878,0.008868,0.008868,0.008868,0.036674,0.117374,0.655692,0.689690,0.081146,0.081146,0.169720,0.443784
3,0.008878,0.008868,0.008868,0.008868,0.036674,0.117374,0.655692,0.689690,0.081146,0.081146,0.169720,0.443784
4,0.008878,0.008868,0.008868,0.008868,0.036674,0.117374,0.655692,0.689690,0.081146,0.081146,0.169720,0.443784
5,0.030450,0.030415,0.030415,0.030415,0.125783,2.484028,5.230828,0.422742,0.023659,0.023659,0.843239,1.633270
6,0.093927,0.093818,0.093818,0.093818,2.577331,1.241795,0.061976,0.137047,0.007670,0.007670,0.440887,0.930842
7,0.540132,1.853533,0.539510,1.853533,2.231213,0.140036,0.010777,0.023832,0.001334,0.001334,0.719523,1.247803
8,0.838898,1.190667,0.839865,1.190667,0.203080,0.063452,0.004883,0.010799,0.000604,0.000604,0.434352,0.862357
OutFit Item,0.191561,0.355342,0.170361,0.355342,0.584445,0.480188,0.866512,0.884994,0.542965,0.155260,NaN,NaN


EXPECTED VALUES FROM LAST ITERATION


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,0,1,2,3,4,5,6,7,8,9,Final Person Ability Logit,Person Standard Error
0,0.999405,0.999406,0.999406,0.999406,0.997546,0.992189,0.907204,0.955788,0.547487,0.547487,4.714737,1.255400
1,0.997557,0.997559,0.997559,0.997559,0.989983,0.968632,0.703838,0.840133,0.227268,0.227268,3.285932,1.159319
2,0.991200,0.991210,0.991210,0.991210,0.964624,0.894955,0.396023,0.591824,0.075055,0.075055,1.975558,1.130398
3,0.991200,0.991210,0.991210,0.991210,0.964624,0.894955,0.396023,0.591824,0.075055,0.075055,1.975558,1.130398
4,0.991200,0.991210,0.991210,0.991210,0.964624,0.894955,0.396023,0.591824,0.075055,0.075055,1.975558,1.130398
5,0.970450,0.970483,0.970483,0.970483,0.888270,0.712976,0.160492,0.297132,0.023112,0.023112,0.711078,1.113012
6,0.914138,0.914228,0.914228,0.914228,0.720462,0.446071,0.058359,0.120529,0.007611,0.007611,-0.450090,1.032364
7,0.649295,0.649557,0.649557,0.649557,0.309481,0.122834,0.010662,0.023277,0.001332,0.001332,-2.233522,0.888055
8,0.456196,0.456482,0.456482,0.456482,0.168800,0.059666,0.004860,0.010683,0.000604,0.000604,-3.031165,0.910814
Final Item Difficulty Logit,-2.843702,-2.844695,-2.844695,-2.844695,-1.402945,-0.212222,2.393425,1.590716,4.504406,4.504406,NaN,NaN
